# Create a Pinecone index with a company description + Twitter Bio
We will use the index as the backend database for our search API

## Install dependencies

In [38]:
pip install -qU sentence-transformers pinecone-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


## Load the Sentence Transformer model

## Test a few different transformer models
We recommend focusing on cosine similarity but feel free to try a couple with examples below and check the similarity score you get to pick the right model

In [39]:
from sentence_transformers import SentenceTransformer, util
# encoding = 'msmarco-distilbert-base-v4' # Similarity: tensor([[0.2574]])
encoding = 'bert-base-nli-mean-tokens' # Similarity: tensor([[0.4602]])
# encoding = 'paraphrase-multilingual-mpnet-base-v2' # Similarity: tensor([[0.4039]])
# encoding = 'all-MiniLM-L6-v2' # Similarity: tensor([[0.3404]])
# encoding = 'all-mpnet-base-v2' # Similarity: tensor([[0.4056]])
# encoding = 'all-roberta-large-v1' # Similarity: tensor([[0.3546]])
model = SentenceTransformer(encoding) 

query_embedding = model.encode('Account-Based Marketing')
passage_embedding = model.encode('6sense is an AI powered account based orchestration platform that provides predictive intelligence to help B2B companies reach their ideal customers at the right time With 6senses solution marketers can identify and prioritize accounts that are most')

print("Similarity:", util.cos_sim(query_embedding, passage_embedding))

Similarity: tensor([[0.4602]])


In [40]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

## Encode a set of company descriptions

In [41]:
# load the data from the csv file
import pandas as pd
df_all = pd.read_csv('../data/tech_companies.csv')


# replace NaN with empty string
df_all = df_all.fillna('')

# remove the rows with no name
df_all = df_all[df_all['name'] != '']

# remove the rows with no description and no twitter__bio
df_all = df_all[(df_all['description'] != '') | (df_all['twitter__bio'] != '')]

# get the number of records in the dataframe
print(len(df_all))

# put the first 20000 companies into a dataframe and only keep the name and description columns
# combine the description and twitter__bio column into a new column called description
df = df_all[:100000][['name', 'description', 'twitter__bio']]
df['description'] = df['description'] + ' ' + df['twitter__bio']
df = df[['name', 'description']]
df.head()


98840


,name,description
0,10x,Your Data Is Your Busness
2,Funsales,Conhea todos os nossos apps Criador de Promoes...
3,Unitedprofile,Nordens kraftfullaste webbportal fr profil yrk...
4,Neurons-IT,Neuronsit is an professional website designin...
5,Church Base,Church base offers a complete church engagemen...


## Push embeddings to Pinecone to be accessed by our endpoint in the future

In [42]:
# get our pinecone_api_key from the environment variable
import os
from dotenv import find_dotenv, load_dotenv

# Load environment variables from .env file
load_dotenv(find_dotenv())

# Set API credentials as variables => Please make sure you have a .env file with the following variables
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENV = os.environ["PINECONE_ENV"]

# connect to our pinecone instance
from pinecone import init
init(
    api_key=PINECONE_API_KEY, 
    environment=PINECONE_ENV
)

# Create a pincone index
# if the index already exists, delete it and create a new one
from pinecone import create_index, list_indexes, Index, delete_index
if "companies" not in list_indexes():
    create_index(name="companies", metric="cosine", shards=1, dimension=768)
else:
    delete_index("companies")
    create_index(name="companies", metric="cosine", shards=1, dimension=768)

# list_indexes()
index = Index("companies")


## Index the Vectors
Since we're likely going to run this on decently big batches, we want to process in chunks

In [43]:
from tqdm.auto import tqdm

# we will use batches of 128
batch_size = 128

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = model.encode(batch['description'].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

100%|██████████| 773/773 [7:30:33<00:00, 34.97s/it]  


{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 98840}},
 'total_vector_count': 98840}

## Test the index by querying it

In [45]:
query = "platform is a web-based designing, prototyping, and collaboration tool, enabling developers, product managers, and marketers to simplify design workflows"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(xq, top_k=3, include_metadata=True)
xc

{'matches': [{'id': '69334',
              'metadata': {'description': 'A bestinclass construction platform '
                                          'that intelligently collates your '
                                          'documentation process to ease of '
                                          'use From BQ etendering and project '
                                          'management BuildSpace is a '
                                          'web-enabled total solution built to '
                                          'cater a variety of functions '
                                          'required in Building and '
                                          'Construction Industry.',
                           'name': 'BuildSpace'},
              'score': 0.915593803,
              'values': []},
             {'id': '91575',
              'metadata': {'description': 'We help convert your tech ideas '
                                          'into reality by using our